In [1]:
from tqdm import tqdm
from torchvision import datasets, transforms
import torch
from torch.utils.data import DataLoader
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torchvision.utils import save_image
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

In [2]:
def show_tensor_images(image_tensor, size=(3, 256, 256)):
    image_unflat = image_tensor.detach().cpu()
    image_grid = make_grid(image_unflat[:], nrow=4)
    plt.axis('off')
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'googlenet', pretrained=True)
model.eval()

Using cache found in /home/mohammaddallash/.cache/torch/hub/pytorch_vision_v0.10.0
/home/mohammaddallash/anaconda3/envs/dl/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/mohammaddallash/anaconda3/envs/dl/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [5]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.RandomRotation(degrees=10),  # Randomly rotate the image by degrees within (-10, 10)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [6]:

train_path = 'Teeth_Dataset/Training'
test_path = 'Teeth_Dataset/Testing'
valid_path = 'Teeth_Dataset/Validation'

# Load datasets for training, testing, and validation
train_dataset = datasets.ImageFolder(root=train_path, transform=preprocess)
test_dataset = datasets.ImageFolder(root=test_path, transform=preprocess)
valid_dataset = datasets.ImageFolder(root=valid_path, transform=preprocess)

# Create DataLoaders for training, testing, and validation
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

In [7]:
num_classes = len(train_dataset.classes)  # Assuming train_dataset is your dataset
model.fc = nn.Linear(model.fc.in_features, num_classes)

In [8]:
for param in model.parameters():
    param.requires_grad = False
model.fc.weight.requires_grad = True
model.fc.bias.requires_grad = True

In [9]:
model = model.to(device)

In [10]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [11]:
num_epochs = 50

for epoch in range(num_epochs):
    epoch_loss = 0.0
    correct = 0
    total = 0

    if(epoch == 25):
      for name, param in model.named_parameters():
          if (name.startswith('inception5')):
            param.requires_grad = True

    # Wrap train_loader with tqdm
    for batch_inputs, batch_targets in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
        batch_inputs, batch_targets = batch_inputs.to('cuda'), batch_targets.to('cuda')

        # Forward pass
        outputs = model(batch_inputs)
        loss = criterion(outputs, batch_targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()

        epoch_loss += loss.item()

        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Average Train Loss: {epoch_loss/len(train_loader):.4f}')

    # Validation phase
    with torch.no_grad():
        for batch_inputs, batch_targets in tqdm(valid_loader, desc=f'validating Epoch {epoch+1}/{num_epochs}'):
            batch_inputs, batch_targets = batch_inputs.to(device), batch_targets.to(device)

            # Forward pass
            outputs = model(batch_inputs)
            _, predicted = torch.max(outputs, 1)
            total += batch_targets.size(0)
            correct += (predicted == batch_targets).sum().item()

    # Calculate accuracy
    accuracy = 100 * correct / total

    print(f'Epoch [{epoch+1}/{num_epochs}], Average Train Loss: {epoch_loss/len(train_loader):.4f}, Validation Accuracy: {accuracy:.2f}%')

Epoch 1/150: 100%|██████████| 49/49 [00:09<00:00,  5.35it/s]


Epoch [1/150], Average Train Loss: 1.5493


validating Epoch 1/150: 100%|██████████| 17/17 [00:02<00:00,  5.67it/s]


Epoch [1/150], Average Train Loss: 1.5493, Validation Accuracy: 57.88%


Epoch 2/150: 100%|██████████| 49/49 [00:09<00:00,  5.30it/s]


...

Epoch [63/150], Average Train Loss: 0.0060


validating Epoch 63/150: 100%|██████████| 17/17 [00:02<00:00,  5.72it/s]


Epoch [63/150], Average Train Loss: 0.0060, Validation Accuracy: 98.15%


Epoch 64/150:   2%|▏         | 1/49 [00:00<00:18,  2.65it/s]


KeyboardInterrupt: 